In [1]:
!wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz

--2020-03-22 23:41:49--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.8.134
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.8.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872478735 (832M) [application/x-gzip]
Saving to: ‘amazon_reviews_us_Sports_v1_00.tsv.gz’

amazon_reviews_us_S 100%[===================>] 832.06M  89.2MB/s    in 9.0s    

2020-03-22 23:42:03 (92.2 MB/s) - ‘amazon_reviews_us_Sports_v1_00.tsv.gz’ saved [872478735/872478735]



In [0]:
!gzip -d amazon_reviews_us_Sports_v1_00.tsv.gz

In [3]:
!apt-get install openjdk-8-jdk -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [4]:
!pip install pyspark

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 56.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=adcc91a19b47e8e7860e5d6374642a2d4fe715d0f489e1d17f71b73480822c54
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
import pandas as pd
import numpy as np
import seaborn as sn
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StructType, StructField, StringType
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler

In [6]:
df = pd.read_csv('amazon_reviews_us_Sports_v1_00.tsv', sep='\t', encoding='utf-8', error_bad_lines=False)

b'Skipping line 7549: expected 15 fields, saw 22\nSkipping line 27856: expected 15 fields, saw 22\nSkipping line 53399: expected 15 fields, saw 22\n'
b'Skipping line 76073: expected 15 fields, saw 22\nSkipping line 89612: expected 15 fields, saw 22\nSkipping line 93502: expected 15 fields, saw 22\nSkipping line 96533: expected 15 fields, saw 22\nSkipping line 107777: expected 15 fields, saw 22\nSkipping line 114290: expected 15 fields, saw 22\nSkipping line 118937: expected 15 fields, saw 22\n'
b'Skipping line 175061: expected 15 fields, saw 22\nSkipping line 185159: expected 15 fields, saw 22\nSkipping line 191448: expected 15 fields, saw 22\n'
b'Skipping line 217523: expected 15 fields, saw 22\nSkipping line 217618: expected 15 fields, saw 22\nSkipping line 238480: expected 15 fields, saw 22\nSkipping line 242797: expected 15 fields, saw 22\nSkipping line 258855: expected 15 fields, saw 22\n'
b'Skipping line 274223: expected 15 fields, saw 22\nSkipping line 284977: expected 15 fields

In [0]:
#Subset
df = df.loc[0:1000000]

In [8]:
df.shape

(1000001, 15)

In [10]:
df.dtypes

marketplace           object
customer_id            int64
review_id             object
product_id            object
product_parent         int64
product_title         object
product_category      object
star_rating           object
helpful_votes        float64
total_votes          float64
vine                  object
verified_purchase     object
review_headline       object
review_body           object
review_date           object
dtype: object

In [11]:
df['star_rating'].unique()

array(['5', '1', '3', '4', '2', '2015-08-26', nan, 3, 5, 4, 1, 2,
       '2015-08-11', '2015-07-09', '2015-06-07', '2015-05-31',
       '2015-05-27', '2015-05-25', '2015-05-20', '2015-05-04',
       '2015-04-18'], dtype=object)

In [0]:
df = df.loc[df['star_rating'].isin([1,2,3,4,5,'1','2','3','4','5'])]
df['star_rating'] = df[['star_rating']].astype(int)

In [13]:
df['star_rating'].value_counts()

5    657534
4    153615
1     72827
3     72040
2     43972
Name: star_rating, dtype: int64

In [0]:
category = df.groupby("product_id")["product_id"].count()

In [15]:
category.head(10)

product_id
0201593246    4
0500615594    1
052800607X    8
0615329020    2
0615389953    1
0615614604    1
0615637620    7
061586628X    3
0692250573    1
0743273575    6
Name: product_id, dtype: int64

In [0]:
spark = SparkSession.builder\
       .appName("Simple recommendation engine using Spark MLlib")\
       .config("spark.some.config.option", "config-value")\
       .getOrCreate()

In [0]:
ratings_df = spark.read.csv("amazon_reviews_us_Sports_v1_00.tsv", sep='\t',header=True)

In [18]:
ratings_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [25]:
ratings_df.select("customer_id",'review_id','product_id','product_title','product_category','star_rating','review_headline','review_body','review_date').show(5)

+-----------+--------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-----------+
|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|Chicago Blackhawk...|          Sports|          5|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|    5782091|R32M0YEWV77XG8|B001GQ3VHG|Copag Poker Size ...|          Sports|          5|       Shipped fast.|These are the bes...| 2015-08-31|
|   45813853| RR8V7WR27NXJ5|B008VS8M58|Baoer 223 5.56x45...|          Sports|          1|Good idea if it w...|It looks good, th...| 2015-08-31|
|    1593730|R1MHO5V9Z932AY|B005F06F4U|All Terrain Tackl...|          Sports|          5|          Five Stars|           Great jig| 2015

In [20]:
ratings_df.show(3)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [0]:
ratings_df.registerTempTable("value")
query = """
SELECT 
  marketplace, 
  customer_id, 
  review_id, 
  product_id, 
  product_parent, 
  product_title, 
  product_category, 
  cast(star_rating as int) as star_rating, 
  helpful_votes, 
  total_votes, 
  vine, 
  verified_purchase, 
  review_headline, 
  review_body, 
  review_date
FROM value as t1
where star_rating in (1,2,3,4,5,'1','2','3','4','5')
limit 1000000
"""
ratings_df = spark.sql(query)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [0]:
ratings_df.registerTempTable("value")
query = """
SELECT 
  cast(customer_id as int) as customer_id, 
  cast(product_id as int) as product_id, 
  star_rating,
  review_date
FROM value as t1
where cast(customer_id as int) is not null and
      cast(product_id as int)  is not null and
      star_rating              is not null and
      review_date              is not null
"""
ratings = spark.sql(query)

In [0]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [0]:
als = ALS(maxIter=20, regParam=0.01, userCol="customer_id", itemCol="product_id", ratingCol="star_rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [31]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.5919395685195923


In [0]:
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
productRecs = model.recommendForAllItems(10)

In [33]:
productRecs.show()

+----------+--------------------+
|product_id|     recommendations|
+----------+--------------------+
| 975780743|[[47331168, 4.997...|
|1427651272|[[35422133, 4.997...|
|1616082828|[[27971424, 3.996...|
| 615614604|[[50615047, 4.997...|
|1934653764|[[26638037, 3.996...|
|1594393761|[[38176138, 4.998...|
| 989448088|[[46040126, 0.989...|
|1931676003|[[15133130, 4.997...|
|1300966955|[[15552424, 4.997...|
|1553602080|[[15255475, 5.000...|
| 983609284|[[10798467, 5.000...|
| 972639500|[[10718919, 3.996...|
| 982123272|[[24392096, 4.998...|
| 875682030|[[33704387, 4.997...|
|1553602102|[[15255475, 4.996...|
|1402734484|[[20181159, 4.997...|
| 201593246|[[15265695, 4.998...|
|1892524031|[[2607490, 4.9972...|
|1553602153|[[15255475, 4.996...|
|1607302438|[[14711022, 4.997...|
+----------+--------------------+
only showing top 20 rows

